# XGBoost

In [1]:
import xgboost as xgb
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

In [4]:
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))

In [5]:
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))

Cargo el df con los features.

In [6]:
%run $merge_features

Cargo los sets de entrenamiento, testing y predicción.

In [7]:
%run $set_de_entrenamiento_testing_y_prediccion

Entreno.

In [8]:
training = labels_training.merge(df_features, how='inner', on='person')

In [9]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count
person,,,,,,,,,,,,,,,
0566e9c1,0,568.000000,320.000000,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0,68
abe7a2fb,0,640.000000,360.000000,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0,96
87ed62de,0,864.000000,1152.000000,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,17
db2c4d27,1,640.491228,429.140351,3.0,31.0,3.0,1.0,166.0,0.0,92.0,13.0,20.0,121.0,114.0,564
cde431db,0,427.000000,320.000000,2.0,1.0,4.0,1.0,0.0,0.0,1.0,2.0,2.0,8.0,3.0,24


In [10]:
columns = list(training.columns)
columns.remove('label')

In [11]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

Hiperparámetros:

In [12]:
param = {'max_depth': 4, 'eta': 0.54, 'gamma': 7, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [13]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [14]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [15]:
test_matrix = xgb.DMatrix(labels_test.loc[:, columns])

In [16]:
labels_test['label_predicted'] = bst.predict(test_matrix)

Métrica de la predicción:

In [17]:
%run $calcular_auc

auc score: 0.8410368506385278
